#### Configuration

In [1]:
cfg = {
    "uploadDir": "./data/corpora/state_of_the_union_address",
    "resultDir": "./data/results",
    "optimal_sentence_length": 16,
}

indicatorsTemplate = {
    "parsable": None,
    "confidence_tokenizer": None,
    "confidence_pos": None,
    "confidence_ner": None,
    "confidence_chunker": None,
    "fit": None,
    "spelling_mistakes": None,
    "avg_sentence_len": None,
    "perc_lowercase": None,
    "perc_uppercase": None,
    "lexical_diversity": None,
    "recognized_by_pos": None,
    "acronyms": None,
    "present_in_dictionary": None,
    "readability_cli": None,
    "readability_ari": None,
    "test_avg_sentence_len": None,
    "test_ari":None,
    "test_cli":None}

#### Initialization

In [2]:
from http import server
import copy
import os
from os import listdir
from os.path import isfile, join
import random
from subprocess import check_output
from threading import Thread
import time
import string
import nltk
import enchant
from spello.model import SpellCorrectionModel
import re
import textstat
import pandas as pd

In [3]:
sp = SpellCorrectionModel(language='en')
# sp.load('./spello_model/en_large.pkl')
sp.load('./spello_model/en_large.pkl')

c:\Users\lbiondo\Anaconda3\envs\tesi\lib\site-packages\spello\model.py:301: UserWarning: This model was saved on spell<1.3.0. As such due to a bug in previous versions, none of customisations made to the config at the time of training were saved along with the model. It is recommended to load the model, apply all required customizations to config and save it again. E.g.

from spello.model import SpellCorrectionModel 
sp = SpellCorrectionModel(language='en')  
sp.load('/home/ubuntu/model.pkl')
sp.config.min_length_for_spellcorrection = 4 # default is 3
sp.config.max_length_for_spellcorrection = 12 # default is 15
sp.save(model_save_dir='/home/ubuntu/')

After this the model will load without any warnings

  warnings.warn("This model was saved on spell<1.3.0. As such due to a bug in previous versions, "


In [4]:
#list where I'm going to save the indicators for each filename
files = dict()

In [5]:
# helper function
def removePunctuationFromTokenized(contentsTokenized):
    excludePuncuation = set(string.punctuation)

    # manually add additional punctuation to remove
    doubleSingleQuote = '\'\''
    doubleDash = '--'
    doubleTick = '``'

    excludePuncuation.add(doubleSingleQuote)
    excludePuncuation.add(doubleDash)
    excludePuncuation.add(doubleTick)

    filteredContents = [
        word for word in contentsTokenized if word not in excludePuncuation]
    return filteredContents

In [6]:
def computeSpellingMistakes(filename, indicator):
    with open( os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        text_tokenized = removePunctuationFromTokenized(
            nltk.word_tokenize(raw_text))
        if len(text_tokenized) == 0:
            result = 0
        else:
            corrected = sp.spell_correct(raw_text)
            mistakes = 0
            for w in text_tokenized:
                if(w in corrected['correction_dict']):
                    mistakes += 1        
            result = (1 - (mistakes / len(text_tokenized)))*100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [7]:
def computeRecognizedByPOS(filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        text_tokenized = removePunctuationFromTokenized(
            nltk.word_tokenize(raw_text))
        if len(text_tokenized) == 0:
            result = 0
        else:    
            text_tagged = nltk.pos_tag(text_tokenized, tagset='universal')
            unknown = 0
            for t in text_tagged:
                if t[1] == "X":
                    unknown += 1
            result = (1 - (unknown/len(text_tagged)))*100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [8]:
def wordcount(s):
    """Split sentence s on punctuation
    and return number of non-empty words
    """
    punct = r"\W"  # non-word characters
    return len([w for w in re.split(punct, s) if w])

def computeAvgSentLen(filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        terminating_punct = "[!?.]"
        sentences = [
            s.strip()  # without trailing whitespace
            for s in re.split(
                terminating_punct,
                "".join(raw_text).replace("\n", " "),  # text as 1 string
            )
            if s.strip()  # non-empty
        ]
        # map each sentece to its wordcount then sum all the wordcounts
        avgSentenceLength = sum(map(wordcount, sentences)) / len(sentences)
        optimalSentenceLen = cfg["optimal_sentence_length"]
        if avgSentenceLength > 2*optimalSentenceLen:
            avgSentenceLength = 2*optimalSentenceLen
        result = (1 - abs(optimalSentenceLen - avgSentenceLength) /
                  optimalSentenceLen) * 100
        files[filename][indicator] = str(result)[0:4]
        #this line to get the actual avg sentence lenght not the score
        files[filename]["test_avg_sentence_len"] = str(avgSentenceLength)
        f.close()

In [9]:
def computePresentInDictionary(filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        text_tokenized = removePunctuationFromTokenized(
            nltk.word_tokenize(raw_text))
        if len(text_tokenized) == 0:
            result = 0
        else:    
            d = enchant.Dict("en_US")
            correct = 0
            for word in text_tokenized:
                if d.check(word):
                    correct += 1
            result = (correct / len(text_tokenized))*100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [10]:
def computeLexicalDiversity(filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        text_tokenized = removePunctuationFromTokenized(nltk.word_tokenize(raw_text))

        if len(text_tokenized) == 0:
            result = 0
        else:
            result = (len(set(text_tokenized)) / len(text_tokenized))*100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [11]:
def setJavaIndicators(filename, result):
    files[filename]["parsable"] = result[0][0:4]
    files[filename]["confidence_tokenizer"] = result[1][0:4]
    files[filename]["confidence_pos"] = result[2][0:4]
    files[filename]["confidence_ner"] = result[3][0:4]
    files[filename]["confidence_chunker"] = result[4][0:4]

def computeJavaIndicators(filename):
    # get the absolute path of the file to pass as argument to jar
    path = os.path.abspath(os.path.join(cfg["uploadDir"], filename))
    pathModels = os.path.abspath("./java-indicators/models")
    # launch java jar
    result = check_output(
        ['java', '-jar', './java-indicators/java-indicators.jar', path, pathModels])
    setJavaIndicators(filename, result.decode().split(","))

In [12]:
def computeAcronyms(filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        text_tokenized = removePunctuationFromTokenized(
            nltk.word_tokenize(raw_text))
        if len(text_tokenized) == 0:
            result = 0
        else:    
            acronym_list = re.findall(r"\b(?:[0-9]+[A-Z][A-Z0-9]*)|(?:[A-Z][A-Z0-9]+)\b|\b[A-Z\.]{2,}\b", raw_text)
            #to remove upper case words present in dictionary from the list of acronyms
            d = enchant.Dict("en_US")
            for acronym in acronym_list:
                if d.check(acronym.lower()):
                    acronym_list.remove(acronym)
            
            acronyms_count = 0
            for word in text_tokenized:
                if word in acronym_list:
                    acronyms_count += 1
            result = (1-(acronyms_count / len(text_tokenized)))*100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [13]:
def computeReadabilityCli( filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        score = textstat.coleman_liau_index(raw_text)
        files[filename]["test_cli"] = str(score)
        optimalScore = 3
        worstScore = 18

        if(score > worstScore):
            score = worstScore

        result = (1 - abs(optimalScore - score) /
                  (worstScore - optimalScore)) * 100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [14]:
def computeReadabilityAri(filename, indicator):
    with open(os.path.join(cfg["uploadDir"], filename), "r",errors='backslashreplace') as f:
        raw_text = f.read()
        score = textstat.automated_readability_index(raw_text)
        files[filename]["test_ari"] = str(score)
        optimalScore = 3
        worstScore = 18

        if(score > worstScore):
            score = worstScore

        result = (1 - abs(optimalScore - score) /
                  (worstScore - optimalScore)) * 100
        files[filename][indicator] = str(result)[0:4]
        f.close()

In [15]:
from IPython.display import clear_output
listOfFileNames = [fileName for fileName in listdir(cfg["uploadDir"]) if isfile(join(cfg["uploadDir"], fileName))]

i = 1
totalFiles = len(listOfFileNames)
for filename in listOfFileNames:
    clear_output(wait=False)
    print(f"Analyzing {filename} | {str(i)} of {totalFiles}")
    i += 1
    #populate the dictionary
    files[filename] = copy.deepcopy(indicatorsTemplate)
    computeJavaIndicators(filename)
    computeSpellingMistakes(filename,"spelling_mistakes")
    computeAvgSentLen(filename,"avg_sentence_len")
    computeLexicalDiversity(filename,"lexical_diversity")
    computeRecognizedByPOS(filename, "recognized_by_pos")
    computeAcronyms(filename, "acronyms")
    computePresentInDictionary(filename,"present_in_dictionary")
    computeReadabilityCli(filename,"readability_cli")
    computeReadabilityAri(filename,"readability_ari")
    

Analyzing Wilson_1920.txt | 227 of 227


In [16]:
listOfFileNames = [fileName for fileName in listdir(cfg["uploadDir"]) if isfile(join(cfg["uploadDir"], fileName))]

indicatorsList = []

for f in listOfFileNames:
    indicatorsList.append(list(files[f].values()))

In [17]:
ind_df = pd.DataFrame(indicatorsList, columns=list(indicatorsTemplate.keys()))
ind_df.replace({'100.': '100'}, regex=True, inplace=True)
ind_df = ind_df.astype(float)
ind_df

,parsable,confidence_tokenizer,confidence_pos,confidence_ner,confidence_chunker,fit,spelling_mistakes,avg_sentence_len,perc_lowercase,perc_uppercase,lexical_diversity,recognized_by_pos,acronyms,present_in_dictionary,readability_cli,readability_ari,test_avg_sentence_len,test_ari,test_cli
0,96.7,99.7,90.2,99.4,90.8,NaN,99.5,0.0,NaN,NaN,35.2,99.9,100.0,99.9,34.0,0.0,32.000000,28.5,12.90
1,96.4,99.7,90.7,99.5,91.3,NaN,99.4,0.0,NaN,NaN,37.2,100.0,100.0,99.6,34.4,0.0,32.000000,34.3,12.84
2,94.8,99.8,92.0,99.7,91.6,NaN,99.4,0.0,NaN,NaN,39.4,100.0,100.0,100.0,31.3,0.0,32.000000,33.2,13.30
3,95.1,99.8,90.9,99.7,90.8,NaN,99.5,0.0,NaN,NaN,40.5,100.0,100.0,99.9,32.4,0.0,32.000000,31.5,13.13
4,97.2,99.7,91.1,99.6,91.5,NaN,99.4,0.0,NaN,NaN,23.6,100.0,99.9,99.7,36.3,0.0,32.000000,32.3,12.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,96.6,99.8,92.0,99.6,90.5,NaN,99.4,0.0,NaN,NaN,33.8,100.0,99.9,99.7,32.0,0.0,32.000000,50.8,13.19
223,95.6,99.8,90.2,99.4,90.4,NaN,99.4,21.7,NaN,NaN,28.2,100.0,99.9,99.8,52.2,0.0,28.521739,20.5,10.17
224,96.6,99.7,90.8,99.5,90.4,NaN,99.5,0.0,NaN,NaN,26.8,100.0,99.9,99.7,46.0,0.0,32.000000,22.0,11.10
225,96.4,99.8,90.7,99.5,90.6,NaN,99.6,15.7,NaN,NaN,29.2,100.0,99.9,99.7,37.9,0.0,29.475309,20.1,12.31


In [18]:
# remove the 100. strings
ind_df.replace({'100.': '100'}, regex=True, inplace=True)
ind_df = ind_df.astype(float)

In [19]:
# Average of each column using DataFrame.mean()
df2 = ind_df.mean(axis=0).to_frame()
df2 = df2.dropna()
df2 = df2.T
df2

,parsable,confidence_tokenizer,confidence_pos,confidence_ner,confidence_chunker,spelling_mistakes,avg_sentence_len,lexical_diversity,recognized_by_pos,acronyms,present_in_dictionary,readability_cli,readability_ari,test_avg_sentence_len,test_ari,test_cli
0,97.244053,99.611454,90.463436,99.291189,91.081498,99.529075,33.360661,26.921586,99.96696,99.880617,99.457269,37.929515,3.736344,26.652513,28.003084,12.304626


In [20]:
outputCsvName = cfg["uploadDir"].replace("./data/corpora/","") + ".csv"
outputPath = os.path.join(cfg["resultDir"],outputCsvName)
df2.to_csv(outputPath, index=False)